# IMAGE_CRAWLING

In [44]:
import os
import requests
import time

from PIL import Image
from io import BytesIO

from tqdm import tqdm

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

from webdriver_manager.chrome import ChromeDriverManager

In [45]:
chrome_options = Options()
chrome_options.add_argument('--headless=new')
chrome_options.add_argument('--nosandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)


In [46]:
root_dir = 'vn_news_thumbnail'
os.makedirs(root_dir, exist_ok=True)

n_pages = 2
img_id = 1

In [47]:
for page_idx in range(n_pages):
    main_url = f'https://vietnamnet.vn/thoi-su-page{page_idx}'
    driver.get(main_url)

    imgs_lst_xpath = '//div[@class="topStory-15nd"]/div/div[1]/a/picture/img'
    imgs_tags = driver.find_elements(By.XPATH,imgs_lst_xpath)


    img_urls = [imgs_tag.get_attribute('data-srcset') for imgs_tag in imgs_tags]


    for img_url in img_urls: 

        img_url_resp = requests.get(img_url)

        try:
            img = Image.open(BytesIO(img_url_resp.content))
        except:
            continue
        if img.mode == 'P':
            img = img.convert('RGB')

        img_name =f'IMG_{img_id}.png'
        img_save_path = os.path.join(root_dir, img_name)

        img.save(img_save_path)
        img_id += 1
